In [1]:
import pandas as pd

In [2]:
# spreads outputted by W3_Predictions.py
submission = pd.read_csv('mydata/womens/original_spreads.csv')
submission.head()

,ID,Pred,TeamID_x,TeamID_y
0,2021_3104_3112,5.993467,3112,3104
1,2021_3104_3116,1.601657,3116,3104
2,2021_3104_3124,19.329731,3124,3104
3,2021_3104_3163,19.730778,3163,3104
4,2021_3104_3208,5.993467,3208,3104


In [3]:
# Switches Pred to the (-1 * Pred) if TeamID_x is not the lower ID as Pred needs to be the spread for the lower ID team
def switch_teams(row):
    idlist = row['ID'].split('_')
    spread = row['Pred']
    if row['TeamID_x'] != int(idlist[1]):
        row['Pred'] = -1 * spread
    row['TeamID_x'] = int(idlist[1])
    row['TeamID_y'] = int(idlist[2])
    return row

submission = submission.apply(switch_teams, axis = 1)

In [4]:
# File for names of teams
team_names = pd.read_csv('ncaaw-march-mania-2021/WTeams.csv')[['TeamID', 'TeamName']]
team_names.head()

,TeamID,TeamName
0,3101,Abilene Chr
1,3102,Air Force
2,3103,Akron
3,3104,Alabama
4,3105,Alabama A&M


In [5]:
# Seeds for 2021
seeds = pd.read_csv('ncaaw-march-mania-2021/WNCAATourneySeeds.csv').query('Season == 2021')[['TeamID', 'Seed']]
seeds.head()

,TeamID,Seed
1408,3390,W01
1409,3257,W02
1410,3208,W03
1411,3116,W04
1412,3283,W05


In [6]:
# join names with seeds
team_names_seeds = pd.merge(team_names, seeds, on = 'TeamID')
team_names_seeds.head()

,TeamID,TeamName,Seed
0,3104,Alabama,X07
1,3112,Arizona,Y03
2,3116,Arkansas,W04
3,3124,Baylor,Z02
4,3125,Belmont,Y12


In [7]:
# Tournament slots for 2021
slots = pd.read_csv('ncaaw-march-mania-2021/WNCAATourneySlots.csv')[['Slot', 'StrongSeed', 'WeakSeed']]
slots.head()

,Slot,StrongSeed,WeakSeed
0,R1W1,W01,W16
1,R1W2,W02,W15
2,R1W3,W03,W14
3,R1W4,W04,W13
4,R1W5,W05,W12


In [8]:
# join slots with names and seeds
team_slots = pd.merge(slots, team_names_seeds, left_on = ['StrongSeed'], right_on = ['Seed'])
team_slots = pd.merge(team_slots, team_names_seeds, left_on = ['WeakSeed'], right_on = ['Seed']).drop(columns = ['Slot', 'StrongSeed', 'WeakSeed', 'Seed_x', 'Seed_y'])
team_slots.head()

,TeamID_x,TeamName_x,TeamID_y,TeamName_y
0,3390,Stanford,3430,Utah Valley
1,3257,Louisville,3265,Marist
2,3208,Georgia,3180,Drexel
3,3116,Arkansas,3460,Wright St
4,3283,Missouri St,3413,UC Davis


In [9]:
# function to switch slots so the lower id is in first slot
def switch_slots(row):
    if row['TeamID_x'] > row['TeamID_y']:
        team_id = row['TeamID_x']
        name = row['TeamName_x']
        row['TeamID_x'] = row['TeamID_y']
        row['TeamID_y'] = team_id
        row['TeamName_x'] = row['TeamName_y']
        row['TeamName_y'] = name
    return row

team_slots = team_slots.apply(switch_slots, axis = 1)
team_slots.head()

,TeamID_x,TeamName_x,TeamID_y,TeamName_y
0,3390,Stanford,3430,Utah Valley
1,3257,Louisville,3265,Marist
2,3180,Drexel,3208,Georgia
3,3116,Arkansas,3460,Wright St
4,3283,Missouri St,3413,UC Davis


In [10]:
# merge submission with team slots to look at Round 1 probabilities
submission_r1 = pd.merge(submission, team_slots, on = ['TeamID_x', 'TeamID_y'])
submission_r1

,ID,Pred,TeamID_x,TeamID_y,TeamName_x,TeamName_y
0,2021_3125_3211,-9.023545,3125,3211,Belmont,Gonzaga
1,2021_3133_3400,-14.556498,3133,3400,Bradley,Texas
2,2021_3140_3353,-8.795882,3140,3353,BYU,Rutgers
3,2021_3141_3234,-10.371886,3141,3234,C Michigan,Iowa
4,2021_3180_3208,-18.510085,3180,3208,Drexel,Georgia
5,2021_3195_3276,-6.354300,3195,3276,FL Gulf Coast,Michigan
6,2021_3199_3333,-0.172177,3199,3333,Florida St,Oregon St
7,2021_3210_3372,-8.693838,3210,3372,Georgia Tech,SF Austin
8,2021_3163_3219,27.715984,3163,3219,Connecticut,High Point
9,2021_3226_3246,-4.907177,3226,3246,Idaho St,Kentucky


In [11]:
ids = list(submission_r1['ID'])

#The lines from Draftkings, inputted manually, in order of IDs in submission_r1
vegas_spreads_x = [-10.5, #  belmont v gonzaga
                   -8.5, # bradley v texas
                   -9.5, # byu v rutgers
                   -10.5, # cmu v iowa
                   -19.5, # drexel v georgia
                   -1.5, # fgcu v mich
                   -5.5, # fsu v oregon st
                   -5.5, # gt v sfa
                   40.5, # uconn v high point
                   -5.5, # idaho st v kentucky
                   33.5, # baylor v jackson st
                   -12.5, # lehigh v wvu
                   19.5, # louisville v marist
                   -2.5, # marquette v va tech
                   -32.5, # mercer v south carolina
                   2.5, # iowa st v michigan st
                   34.5, # Maryland v mt st marys
                   -13.5, # MTSU v tenn
                   -20.5, # nc a&t v NC State
                   -1.5, # alabama v unc
                   -4.5, # s dakota st v syracuse
                   3.5, # oregon v south dakota
                   19.5, # arizona v stony brook
                   16.5, # texs a&m v troy
                   9.5, # Missouri st v uc davis
                   3.5, # NW v UCF
                   44.5, # Stanford v Utah Valley
                   16, # Indiana v VCU
                   7.5, # Ok St v WF
                   2, # USF v Wash St
                   7.5, # Arkansas v Wright St
                   20.5 # UCLA v Wyoming
                  ] # the spread for TeamID_x (lower team id)

# make datframe of these odds
spread = pd.DataFrame({'ID': ids, 'Spread_x': vegas_spreads_x})
spread.head()

,ID,Spread_x
0,2021_3125_3211,-10.5
1,2021_3133_3400,-8.5
2,2021_3140_3353,-9.5
3,2021_3141_3234,-10.5
4,2021_3180_3208,-19.5


In [12]:
# merge vegas odds with submission probabilities for round 1
submission_r1 = pd.merge(submission_r1, spread, on = 'ID')

# Calculate absolute difference between my probabilities and Vegas'
submission_r1['AbsDiff'] = abs(submission_r1['Spread_x'] - submission_r1['Pred'])

# print submission_r1 ordering by the difference in win probability
submission_r1.sort_values('AbsDiff')

,ID,Pred,TeamID_x,TeamID_y,TeamName_x,TeamName_y,Spread_x,AbsDiff
19,2021_3104_3314,-1.448886,3104,3314,Alabama,North Carolina,-1.5,0.051114
3,2021_3141_3234,-10.371886,3141,3234,C Michigan,Iowa,-10.5,0.128114
15,2021_3235_3277,2.656765,3235,3277,Iowa St,Michigan St,2.5,0.156765
27,2021_3231_3433,16.368992,3231,3433,Indiana,VCU,16.0,0.368992
9,2021_3226_3246,-4.907177,3226,3246,Idaho St,Kentucky,-5.5,0.592823
2,2021_3140_3353,-8.795882,3140,3353,BYU,Rutgers,-9.5,0.704118
29,2021_3378_3450,2.728433,3378,3450,South Florida,Washington St,2.0,0.728433
4,2021_3180_3208,-18.510085,3180,3208,Drexel,Georgia,-19.5,0.989915
22,2021_3112_3392,18.468507,3112,3392,Arizona,Stony Brook,19.5,1.031493
13,2021_3266_3439,-3.597086,3266,3439,Marquette,Virginia Tech,-2.5,1.097086


In [13]:
# adjusts all spreads for a given team by the given amount of points
# positive adj means team is undervalued by model
# negative adj means team is overvalued by model
def team_adjustment(row, team_id, adj):
    idlist = row['ID'].split('_')
    
    # increase spread for team by adj amount
    if team_id == int(idlist[1]):
        return row['Pred'] + adj
    elif team_id == int(idlist[2]):
        return row['Pred'] - adj
    else:
        return row['Pred']
    
# look at Vegas odds, and see which teams the model undervalues that make sense (injuries, COVID, other reasons)
# and adjust their win probabilities for all games manually using function above

submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 3124	, -2), axis = 1) # Baylor
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 3390, 3), axis = 1) # Stanford
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 3163, 2), axis = 1) # UConn
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 3199, -1.5), axis = 1) # FSU
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 3333, 1.5), axis = 1) # Oregon st
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 3133, 3), axis = 1) # Bradley
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 3460, 2), axis = 1) # Wright St
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 3355, 2), axis = 1) # S Dakota St
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 3377, 1), axis = 1) # S Dakota
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 3195, 2), axis = 1) # FGCU

In [14]:
# run same code to compare new altered probabilities to Vegas odds

# merge submission with team slots to look at Round 1 spreads
submission_r1 = pd.merge(submission, team_slots, on = ['TeamID_x', 'TeamID_y'])

# merge vegas odds with submission spreads for round 1
submission_r1 = pd.merge(submission_r1, spread, on = 'ID')

# Calculate absolute difference between my spreads and Vegas'
submission_r1['AbsDiff'] = abs(submission_r1['Spread_x'] - submission_r1['Pred'])

# print submission_r1 ordering by the difference in spread
submission_r1.sort_values('AbsDiff')

,ID,Pred,TeamID_x,TeamID_y,TeamName_x,TeamName_y,Spread_x,AbsDiff
19,2021_3104_3314,-1.448886,3104,3314,Alabama,North Carolina,-1.5,0.051114
3,2021_3141_3234,-10.371886,3141,3234,C Michigan,Iowa,-10.5,0.128114
15,2021_3235_3277,2.656765,3235,3277,Iowa St,Michigan St,2.5,0.156765
27,2021_3231_3433,16.368992,3231,3433,Indiana,VCU,16.0,0.368992
9,2021_3226_3246,-4.907177,3226,3246,Idaho St,Kentucky,-5.5,0.592823
2,2021_3140_3353,-8.795882,3140,3353,BYU,Rutgers,-9.5,0.704118
29,2021_3378_3450,2.728433,3378,3450,South Florida,Washington St,2.0,0.728433
4,2021_3180_3208,-18.510085,3180,3208,Drexel,Georgia,-19.5,0.989915
22,2021_3112_3392,18.468507,3112,3392,Arizona,Stony Brook,19.5,1.031493
13,2021_3266_3439,-3.597086,3266,3439,Marquette,Virginia Tech,-2.5,1.097086


### Kaggle Alterations

Kaggle allows two submission files, so I will use my spreads for the lopsided games in one submission and use Vegas' spreads in the other

In [16]:
submission1 = submission.copy()[['ID', 'Pred']]
submission2 = submission.copy()[['ID', 'Pred']]

In [15]:
query = 'ID ==\'2021_3163_3219\'' # UConn v Highpoint
query = query + ' or ID ==\'2021_3268_3291\'' # Maryland v Mt st marys
query = query + ' or ID ==\'2021_3273_3376\'' # mercer v south carolina
query = query + ' or ID ==\'2021_3390_3430\'' # stanford v utah valley
query = query + ' or ID ==\'2021_3124_3238\'' # baylor v jackson st
submission.query(query)

,ID,Pred,TeamID_x,TeamID_y
419,2021_3163_3219,29.715984,3163,3219
565,2021_3124_3238,28.947166,3124,3238
831,2021_3273_3376,-25.592896,3273,3376
963,2021_3268_3291,26.129549,3268,3291
1676,2021_3390_3430,38.793786,3390,3430


In [17]:
# make manual adjustments to round 1 win probabilities based on Vegas odds
submission2.loc[419, 'Pred'] = 40.5  # UConn v Highpoint
submission2.loc[963, 'Pred'] = 34.5  # Maryland v Mt st marys
submission2.loc[831, 'Pred'] = -32.5  # mercer v south carolina
submission2.loc[1676, 'Pred'] = 44.5  # stanford v utah valley
submission2.loc[565, 'Pred'] = 33.5 # baylor v jackson st

In [18]:
query = 'ID ==\'2021_3163_3219\'' # UConn v Highpoint
query = query + ' or ID ==\'2021_3268_3291\'' # Maryland v Mt st marys
query = query + ' or ID ==\'2021_3273_3376\'' # mercer v south carolina
query = query + ' or ID ==\'2021_3390_3430\'' # stanford v utah valley
query = query + ' or ID ==\'2021_3124_3238\'' # baylor v jackson st
submission2.query(query)

,ID,Pred
419,2021_3163_3219,40.5
565,2021_3124_3238,33.5
831,2021_3273_3376,-32.5
963,2021_3268_3291,34.5
1676,2021_3390_3430,44.5


In [19]:
# write submissions to csv
submission.to_csv('mydata/womens/altered_spreads.csv', index = False)
submission1.to_csv('mydata/womens/submission1_spread.csv', index = False)
submission2.to_csv('mydata/womens/submission2_spread.csv', index = False)